In [1]:
%use multik

In [2]:
data class Star(
    var id : Int? = null,
    var hip : Int? = null,
    var hd : Int? = null,
    var hr : Int? = null,
    var gl : String? = null,
    var bf : String? = null,
    var proper : String? = null,
    var ra : Double? = null,
    var dec : Double? = null,
    var dist : Double? = null,
    var pmra : Double? = null,
    var pmdec : Double? = null,
    var rv : Double? = null,
    var mag : Double? = null,
    var absmag : Double? = null,
    var spect : String? = null,
    var ci : Double? = null,
    var x : Double? = null,
    var y : Double? = null,
    var z : Double? = null,
    var vx : Double? = null,
    var vy : Double? = null,
    var vz : Double? = null,
    var rarad : Double? = null,
    var decrad : Double? = null,
    var pmrarad : Double? = null,
    var pmdecrad : Double? = null,
    var bayer : String? = null,
    var flam : Int? = null,
    var con : String? = null,
    var comp : Int? = null,
    var comp_primary : Int? = null,
    var base : String? = null,
    var lum : Double? = null,
    var variable : String? = null,
    var var_min : Double? = null,
    var var_max : Double? = null,
)

In [3]:
var typeList: MutableList<String> = mutableListOf("Int?", "Int?", "Int?", "Int?", "String?", "String?", "String?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "String?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "String?", "Int?", "String?", "Int?", "Int?", "String?", "Double?", "String?", "Double?", "Double?")

In [4]:
fun changeType(data: String, type: String): Any?{
    when(type){
        "Int?" -> return data.toIntOrNull()
        "Double?" -> return data.toDoubleOrNull()
        "String?" -> return data
        else -> throw IllegalAccessException()
    }
}

In [5]:
import java.io.BufferedReader
import java.io.File
import java.io.FileReader
import kotlin.reflect.KMutableProperty
import kotlin.reflect.full.memberProperties
import kotlin.streams.toList

fun readCsv(fileName: String, typeList: MutableList<String>): MutableList<Star> {
    val file = File(fileName)
    val reader = BufferedReader(FileReader(file))

    val header = reader.readLine()
    var properList = header.split(",").toMutableList()

    for ((i, item) in properList.withIndex()) {
        if (item == "var") {
            properList[i] = "variable"
        }
    }

    val data = reader.lines().toList()
    var i = 0

    var starList: MutableList<Star> = mutableListOf()
    for (line in data) {
        i++
        var star = Star()
        val stringData = line.split(',')

        for ((i, propertyName) in properList.withIndex()) {
            val property = star::class.memberProperties.find { it.name == propertyName }

            val type = typeList[i]
            val value = changeType(stringData[i], type)

            if (property != null && property is KMutableProperty<*>) {
                property.setter.call(star, value)
            }
        }
        starList.add(star)
    }
    return starList
}
var starList = readCsv("./hyg_v37.csv", typeList)

In [6]:
var new_starList = starList.subList(0, starList.size)

In [7]:
println("ra:${new_starList[1000].rarad} dec:${new_starList[1000].decrad}")

ra:0.054757002 dec:0.429985749


In [8]:
var stars = mutableListOf<List<Double>>()

for(star in new_starList){
    val ra = star.rarad?.toDouble() as Double
    val dec = star.decrad?.toDouble() as Double
    val lis = listOf(cos(ra) * cos(dec), cos(dec) * sin(ra), sin(dec))
    stars.add(lis)
}

var starList = stars.toList()

In [9]:
var starArray =  mk.ndarray(starList)

In [10]:
fun makeLocation(starList: List<Star>): List<List<Double>>{
    var stars = mutableListOf<List<Double>>()

    for(star in new_starList){
        val ra = star.rarad?.toDouble() as Double
        val dec = star.decrad?.toDouble() as Double
        val lis = listOf(cos(ra) * cos(dec), cos(dec) * sin(ra), sin(dec), star.id?.toDouble() as Double, star.mag as Double)
        stars.add(lis)
    }

    var starList = stars.toList()
    return starList
}

In [11]:
var starArray = makeLocation(new_starList)
print(starArray::class.simpleName)

ArrayList

In [12]:
fun getXAxis(theta: Double, phi: Double): List<Double>{
    return listOf(cos(theta) * cos(phi), sin(theta) * cos(phi), sin(phi), 0.0, 0.0)
}

fun getZAxis(theta: Double, phi: Double): List<Double>{
    return getXAxis(theta, phi + PI * 0.5)
}

fun crossProduct(arr1: List<Double>, arr2: List<Double>): List<Double>{
    return listOf(arr1[1] * arr2[2] - arr1[2] * arr2[1], arr1[2] * arr2[0] - arr1[0] * arr2[2], arr1[0] * arr2[1] - arr2[0] * arr1[1], 0.0, 0.0)
}

fun getYAxis(theta: Double, phi: Double): List<Double>{
    return crossProduct(getZAxis(theta, phi), getXAxis(theta, phi))
}

fun getTransformMatrix(theta: Double, phi: Double): NDArray<Double, D2>{
    val new_theta = theta * PI / 180
    val new_phi = phi * PI / 180
    return mk.ndarray(listOf(getXAxis(new_theta, new_phi), getYAxis(new_theta, new_phi), getZAxis(new_theta, new_phi), listOf(0.0, 0.0, 0.0, 1.0, 0.0),listOf(0.0, 0.0, 0.0, 0.0, 1.0)))
}

In [17]:
fun changeCoords(theta: Double, phi: Double, starArray: List<List<Double>>): NDArray<Double, D2>{
    val trans = getTransformMatrix(theta, phi)
    val source = mk.ndarray(starArray)
    val cc = source dot trans.transpose()
    
    return cc
}

val b = changeCoords(60.0, -30.0, starArray)
val cc = b.shape
print("${cc[0]} ${cc[1]}")

119624 5

## 2. 메르카토르 변환

$ x= R(\lambda)$, $y = R \log{(\sec{\theta} + \tan{\theta})}$       

In [27]:
fun rem(_a: Double, _b: Double) : Double{
    val quotient = (_a/_b).toInt()
    return _a - quotient * _b
}

fun getAngle(_sin: Double, _cos: Double) : Double{
    val _tan = _sin / _cos
    var angle = if(_cos > 0.0) atan(_tan) else atan(_tan) + PI
    return rem(angle + 2 * PI, 2 * PI)
}

val mutableList : MutableList<List<Double>> = mutableListOf()

for(i in 0 until cc[0]){
    val x = b[i][0]
    val y = b[i][1]
    val z = b[i][2]
    val ind = b[i][3]
    val mag = b[i][4]
    
    val theta = getAngle(y, x)
    val phi = asin(z)
    
    val new_x = theta
    val new_y = ln((1.0/cos(phi)) + tan(phi))
    mutableList.add(listOf(new_x, new_y, ind, mag))
}

In [28]:
println(mutableList)

[[5.3261497041973165, -0.05342721306316958, 2.0, 9.27], [4.747266647167417, 0.9461929829536166, 3.0, 6.61], [5.603096365404993, -0.5861549678752094, 4.0, 8.06], [5.495253730554546, -0.3925843152965661, 5.0, 8.55], [5.143972598598843, 0.31960712973367916, 6.0, 12.31], [4.994505283872712, 0.5925633930858774, 7.0, 9.64], [4.929221812993898, 0.6981598096201128, 8.0, 9.05], [4.784265800043365, 0.9017125343149597, 9.0, 8.59], [5.59276324667683, -0.5678884577402546, 10.0, 8.59], [4.595608784864357, 1.108793578006751, 11.0, 7.34], [5.45606600144901, -0.3160644858390797, 12.0, 8.43], [5.350238736388986, -0.10198541180349613, 13.0, 8.8], [5.179718015000342, 0.24999370789581005, 14.0, 7.25], [4.508249300154688, 1.1859385095891564, 15.0, 8.6], [5.492280617470161, -0.3856800245596724, 16.0, 8.15], [5.636092887857388, -0.6397616398548653, 17.0, 11.71], [5.2090716877458085, 0.19083585235175707, 18.0, 11.03], [4.757158675910359, 0.9358108997395317, 19.0, 6.53], [4.957105950918869, 0.6555326287748774, 